In [1]:
import requests
from bs4 import BeautifulSoup
import json
import time
from openpyxl import Workbook, load_workbook
from IPython.display import clear_output

# Базовый URL страницы с квартирами
base_url = 'https://krisha.kz/arenda/kvartiry/almaty/?page='

# Количество страниц для парсинга
pages_to_scrape = 100

# Имя файла Excel
excel_file = 'krisha_arenda_data.xlsx'

# Создаем новый Excel файл, если его нет
wb = Workbook()
ws = wb.active
# Добавляем заголовки
columns = [
    'ID', 'Title', 'Address', 'User Type', 'Square', 'Rooms', 'Status', 'Latitude', 'Longitude', 
    'Price per m²', 'Created At', 'Owner Title', 'Owner Label Title', 'Owner Label Name', 
    'Year of Construction', 'Condition', 'Floor', 'Number of Photos', 'Category Alias', 'District', 'Country', 'City', 'Price'
]
ws.append(columns)
wb.save(excel_file)

# Функция для извлечения JSON-данных из JavaScript на странице объявления
def extract_data_from_script(soup):
    script = soup.find('script', string=lambda t: t and 'window.data' in t)
    
    if script:
        script_content = script.string.strip()
        json_data_start = script_content.find('{')
        json_data_end = script_content.rfind('}') + 1
        try:
            data = json.loads(script_content[json_data_start:json_data_end])
            return data
        except json.JSONDecodeError:
            print("Ошибка при декодировании JSON.")
            return None
    return None

# Функция для парсинга детализированной страницы объявления
def parse_detailed_page(data_id):
    detailed_url = f'https://krisha.kz/a/show/{data_id}'
    response = requests.get(detailed_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Извлекаем данные из JavaScript
    data = extract_data_from_script(soup)

    if data:
        advert = data.get('advert', {})
        # Данные с adverts, так как в них могут быть важные данные
        adverts_list = data.get('adverts', [])

        # Извлекаем нужные данные из advert
        advert_id = advert.get('id', 'N/A')
        title = advert.get('title', 'N/A')
        address = advert.get('addressTitle', 'N/A')
        user_type = advert.get('userType', 'N/A')
        square = advert.get('square', 'N/A')
        rooms = advert.get('rooms', 'N/A')
        status = advert.get('status', 'N/A')
        lat = advert.get('map', {}).get('lat', 'N/A')
        lon = advert.get('map', {}).get('lon', 'N/A')
        price = advert.get('price', 'N/A')
        district = advert.get('address', {}).get('district', 'N/A')
        country = advert.get('address', {}).get('country', 'N/A')
        city = advert.get('address', {}).get('city', 'N/A')

        # Дополнительные данные из adverts
        if adverts_list:
            price_m2 = adverts_list[0].get('priceM2', 'N/A')
            created_at = adverts_list[0].get('createdAt', 'N/A')
            owner_data = adverts_list[0].get('owner', {})
            owner_title = owner_data.get('title', 'N/A')
            owner_label_title = owner_data.get('label', {}).get('title', 'N/A')
            owner_label_name = owner_data.get('label', {}).get('name', 'N/A')
            num_photos = adverts_list[0].get('nbPhotos', 'N/A')
            category_alias = adverts_list[0].get('category', {}).get('label', 'N/A')
        else:
            price_m2 = 'N/A'
            created_at = 'N/A'
            owner_title = 'N/A'
            owner_label_title = 'N/A'
            owner_label_name = 'N/A'
            num_photos = 'N/A'
            category_alias = 'N/A'

        # Парсим данные о годе постройки, этаже и состоянии
        construction_year = 'N/A'
        condition = 'N/A'
        floor = 'N/A'

        # Ищем соответствующие теги с нужной информацией
        for item in soup.find_all('div', class_='offer__info-item'):
            data_name = item.get('data-name', '')
            if data_name == 'house.year':
                construction_year = item.find('div', class_='offer__advert-short-info').text.strip()
            elif data_name == 'flat.floor':
                floor = item.find('div', class_='offer__advert-short-info').text.strip()
            elif data_name == 'flat.renovation':
                condition = item.find('div', class_='offer__advert-short-info').text.strip()

        # Сохраняем данные поочередно в Excel
        wb = load_workbook(excel_file)
        ws = wb.active
        row = [
            advert_id, title, address, user_type, square, rooms, status, lat, lon, 
            price_m2, created_at, owner_title, owner_label_title, owner_label_name, 
            construction_year, condition, floor, num_photos, category_alias, district, country, city, price
        ]
        ws.append(row)
        wb.save(excel_file)
        return True
    return False

# Счетчик строк
rows_processed = 0

# Проход по каждой странице списка объявлений
for page in range(1, pages_to_scrape + 1):
    url = base_url + str(page)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Проход по каждому объявлению и извлечение data-id
    for listing in soup.find_all('div', class_='a-card'):
        data_id = listing.get('data-id')
        if data_id:
            success = parse_detailed_page(data_id)
            if success:
                rows_processed += 1
                clear_output(wait=True)
                print(f"Парсинг объявления с ID: {data_id} завершен. Обработано строк: {rows_processed}")
                time.sleep(1)  # Пауза для уменьшения нагрузки на сервер

print(f"Парсинг завершен. Данные сохранены в {excel_file}.")


Парсинг объявления с ID: 697967365 завершен. Обработано строк: 2003
Парсинг завершен. Данные сохранены в krisha_arenda_data.xlsx.
